In [1]:
from itertools import chain
from pathlib import Path

from utils.dataset_utils import get_politicalness_datasets, \
    get_politicalness_datasets_from_leaning_datasets
from utils.model_utils import evaluate_models, get_custom_politicalness_models

In [2]:
GET_DATASETS = lambda: chain(
    get_politicalness_datasets(),
    get_politicalness_datasets_from_leaning_datasets()
)
DATASET_SAMPLE_SIZE = 1_000

datasets = list(
    map(
        lambda dataset: dataset
        .take_even_class_distribution_sample(DATASET_SAMPLE_SIZE)
        .transform_for_inference()
        .to_huggingface(),
        GET_DATASETS(),
    )
)

In [3]:
# Lambda, so that the generator can be reused.
GET_MODELS = lambda: get_custom_politicalness_models(Path("dataset_benchmark") / "leave_one_in" / "politicalness")
TRUNCATE_TOKENS = True

evaluate_models(
    GET_MODELS,
    datasets,
    TRUNCATE_TOKENS
)

evaluating dataset_benchmark/leave_one_out/politicalness/bert-base-cased/amazon_reviews_2023 on:
  article_bias_prediction


  0%|          | 0/1000 [00:00<?, ?it/s]

  commoncrawl_news_articles


  0%|          | 0/1000 [00:00<?, ?it/s]

  dem_rep_party_platform_topics


...

  webis_bias_flipper_18


  0%|          | 0/1000 [00:00<?, ?it/s]

  webis_news_bias_20


  0%|          | 0/1000 [00:00<?, ?it/s]

,article_bias_prediction,commoncrawl_news_articles,dem_rep_party_platform_topics,gpt4_political_bias,gpt4_political_ideologies,media_political_stance,parliament_speeches_2024,political_podcasts,political_tweets,qbias,webis_bias_flipper_18,webis_news_bias_20,average
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/amazon_reviews_2023,999 / 1000 (100 %),851 / 1000 (85 %),974 / 1000 (97 %),612 / 612 (100 %),1000 / 1000 (100 %),945 / 1000 (94 %),989 / 1000 (99 %),986 / 1000 (99 %),972 / 1000 (97 %),979 / 1000 (98 %),988 / 1000 (99 %),993 / 1000 (99 %),11288 / 11612 (97 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/article_bias_prediction,996 / 1000 (100 %),896 / 1000 (90 %),982 / 1000 (98 %),612 / 612 (100 %),1000 / 1000 (100 %),968 / 1000 (97 %),998 / 1000 (100 %),990 / 1000 (99 %),983 / 1000 (98 %),981 / 1000 (98 %),984 / 1000 (98 %),992 / 1000 (99 %),10386 / 10612 (98 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/commoncrawl_news_articles,995 / 1000 (100 %),738 / 1000 (74 %),988 / 1000 (99 %),612 / 612 (100 %),1000 / 1000 (100 %),968 / 1000 (97 %),998 / 1000 (100 %),992 / 1000 (99 %),979 / 1000 (98 %),988 / 1000 (99 %),988 / 1000 (99 %),993 / 1000 (99 %),10501 / 10612 (99 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/dem_rep_party_platform_topics,998 / 1000 (100 %),894 / 1000 (89 %),950 / 1000 (95 %),612 / 612 (100 %),1000 / 1000 (100 %),970 / 1000 (97 %),996 / 1000 (100 %),991 / 1000 (99 %),983 / 1000 (98 %),976 / 1000 (98 %),984 / 1000 (98 %),994 / 1000 (99 %),10398 / 10612 (98 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/dialogsum,998 / 1000 (100 %),895 / 1000 (90 %),985 / 1000 (98 %),612 / 612 (100 %),1000 / 1000 (100 %),958 / 1000 (96 %),999 / 1000 (100 %),989 / 1000 (99 %),977 / 1000 (98 %),976 / 1000 (98 %),981 / 1000 (98 %),996 / 1000 (100 %),11366 / 11612 (98 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/free_news,997 / 1000 (100 %),883 / 1000 (88 %),965 / 1000 (96 %),609 / 612 (100 %),1000 / 1000 (100 %),944 / 1000 (94 %),995 / 1000 (100 %),977 / 1000 (98 %),966 / 1000 (97 %),973 / 1000 (97 %),981 / 1000 (98 %),991 / 1000 (99 %),11281 / 11612 (97 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/goodreads_book_genres,1000 / 1000 (100 %),869 / 1000 (87 %),988 / 1000 (99 %),612 / 612 (100 %),999 / 1000 (100 %),945 / 1000 (94 %),992 / 1000 (99 %),982 / 1000 (98 %),974 / 1000 (97 %),982 / 1000 (98 %),981 / 1000 (98 %),991 / 1000 (99 %),11315 / 11612 (97 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/gpt4_political_bias,988 / 1000 (99 %),738 / 1000 (74 %),961 / 1000 (96 %),595 / 612 (97 %),1000 / 1000 (100 %),908 / 1000 (91 %),979 / 1000 (98 %),905 / 1000 (90 %),968 / 1000 (97 %),939 / 1000 (94 %),969 / 1000 (97 %),987 / 1000 (99 %),10342 / 11000 (94 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/gpt4_political_ideologies,998 / 1000 (100 %),879 / 1000 (88 %),990 / 1000 (99 %),612 / 612 (100 %),975 / 1000 (98 %),965 / 1000 (96 %),999 / 1000 (100 %),993 / 1000 (99 %),983 / 1000 (98 %),988 / 1000 (99 %),988 / 1000 (99 %),996 / 1000 (100 %),10391 / 10612 (98 %)
dataset_benchmark/leave_one_out/politicalness/bert-base-cased/imdb,999 / 1000 (100 %),911 / 1000 (91 %),989 / 1000 (99 %),612 / 612 (100 %),1000 / 1000 (100 %),973 / 1000 (97 %),995 / 1000 (100 %),990 / 1000 (99 %),981 / 1000 (98 %),988 / 1000 (99 %),989 / 1000 (99 %),998 / 1000 (100 %),11425 / 11612 (98 %)
